# ORTOOLS  알고리즘 구현

In [1]:
import pandas as pd
pd.read_csv("../data/final_num_171.csv")
distance_matrix=pd.read_csv("../data/final_distance.csv")

In [2]:

matrix = distance_matrix.values
matrix[[0, 171]] = matrix[[171, 0]]

# 열 교체
matrix[:, [0, 171]] = matrix[:, [171, 0]]
lst=matrix
demands=[0,612, 306, 921, 982, 231, 1068, 173, 151, 35, 612, 122, 122, 140, 414, 45, 182, 224, 122, 306, 45, 45, 136, 224, 133, 60, 140, 61, 538, 136, 758, 224, 196, 61, 45, 820, 97, 35, 61, 285, 97, 224, 232, 232, 98, 98, 52, 605, 97, 133, 140, 140, 140, 97, 136, 97, 136, 136, 97, 133, 98, 61, 215, 125, 136, 136, 94, 94, 136, 280, 94, 97, 173, 1465, 35, 285, 384, 217, 332, 258, 984, 112, 73, 73, 151, 332, 1043, 440, 440, 196, 258, 760, 136, 332, 112, 378, 196, 52, 112, 1010, 151, 71, 950, 160, 378, 432, 280, 605, 210, 35, 35, 35, 125, 217, 173, 71, 322, 322, 436, 479, 1065, 387, 173, 322, 322, 71, 564, 423, 432, 215, 215, 92, 52, 160, 52, 92, 92, 160, 160, 280, 397, 35, 35, 35, 35, 35, 35, 35, 35, 35, 1231, 61, 61, 60, 35, 35, 35, 35, 35, 35, 60, 61, 94, 35, 60, 61, 182, 60, 136, 136, 414, 94]

In [3]:
sum(demands)

40437

In [3]:
print(47000/sum(demands))
demand_lst=[]
for i in demands:
    i=demand_lst.append(int(i*1.17))

1.1623018522640156


In [4]:
sum(demand_lst)-sum([6100, 5000, 3000,1500]*2+[6100,5000,3000,1500,1500])

-1080

In [6]:
len(demands),len(demand_lst)

(172, 172)

In [7]:
#1500
sum(demand_lst)-sum([6100, 5000,3700 , 3000]*2+[5000,3700,3000])
#3000
sum(demand_lst)-sum([6100, 5000,3700 ,1500]*2+[6100,5000,3700])
#3700
sum(demand_lst)-sum([6100, 5000, 3000,1500]*2+[6100,5000,3000,1500,1500])
#5000
sum(demand_lst)-sum([6100,3700,3000,1500]*2+[6100,3700,3000,1500,3700,1500])
#6100
sum(demand_lst)-sum([5000,3700 , 3000,1500]*2+[5000,3700,3000,1500,5000,3000])
print(sum(demand_lst)-sum([6100, 5000,3700 , 3000]*2+[6100,3700,3000]))

-1180


In [16]:
data=create_data_model()
len([6100, 5000,3700 , 3000]*2+[6100,3700,3000])

11

In [5]:
"""Capacited Vehicles Routing Problem (CVRP)."""
import numpy as np 
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = lst
    data["demands"] = demand_lst
    data["vehicle_capacities"] = [6100, 5000, 3000,1500]*2+[6100,5000,3000,1500,1500]
    data["num_vehicles"] = len(data["vehicle_capacities"])
    data["depot"] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    #print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    total_load = 0
    routelst=[]

    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Load({route_load}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        plan_output += f"Load of the route: {route_load}\n"
    #    print(plan_output)
        routelst.append(plan_output)
        total_distance += route_distance
        total_load += route_load
    print(f"Total distance of all routes: {total_distance}m")
    #print(f"Total load of all routes: {total_load}")
    return solution.ObjectiveValue(),total_distance,total_load,routelst

def main(first,local_search):
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        first
        #routing_enums_pb2.FirstSolutionStrategy.SAVINGS
    )
    search_parameters.local_search_metaheuristic = (
        local_search
        #routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(30)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        return (print_solution(data, manager, routing, solution))
    else:
        return "no result","no result","no result","no result"
result=[]

# FirstSolutionStrategy와 LocalSearchMetaheuristic의 문자열과 실제 열거형 값을 매핑하는 딕셔너리 생성
first_solution_strategy_mapping = {
    'GLOBAL_CHEAPEST_ARC': routing_enums_pb2.FirstSolutionStrategy.GLOBAL_CHEAPEST_ARC,
    'LOCAL_CHEAPEST_ARC': routing_enums_pb2.FirstSolutionStrategy.LOCAL_CHEAPEST_ARC,
    'PATH_CHEAPEST_ARC': routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC,
    'EVALUATOR_STRATEGY': routing_enums_pb2.FirstSolutionStrategy.EVALUATOR_STRATEGY,
    'ALL_UNPERFORMED': routing_enums_pb2.FirstSolutionStrategy.ALL_UNPERFORMED,
    'BEST_INSERTION': routing_enums_pb2.FirstSolutionStrategy.BEST_INSERTION,
    'PARALLEL_CHEAPEST_INSERTION': routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION,
    'LOCAL_CHEAPEST_INSERTION': routing_enums_pb2.FirstSolutionStrategy.LOCAL_CHEAPEST_INSERTION,
    'SAVINGS': routing_enums_pb2.FirstSolutionStrategy.SAVINGS,
    'SWEEP': routing_enums_pb2.FirstSolutionStrategy.SWEEP,
    'FIRST_UNBOUND_MIN_VALUE': routing_enums_pb2.FirstSolutionStrategy.FIRST_UNBOUND_MIN_VALUE,
    'CHRISTOFIDES': routing_enums_pb2.FirstSolutionStrategy.CHRISTOFIDES,
}

local_search_metaheuristic_mapping = {
    'GREEDY_DESCENT': routing_enums_pb2.LocalSearchMetaheuristic.GREEDY_DESCENT,
    'GUIDED_LOCAL_SEARCH': routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH,
    'SIMULATED_ANNEALING': routing_enums_pb2.LocalSearchMetaheuristic.SIMULATED_ANNEALING,
    'TABU_SEARCH': routing_enums_pb2.LocalSearchMetaheuristic.TABU_SEARCH,
    'GENERIC_TABU_SEARCH': routing_enums_pb2.LocalSearchMetaheuristic.GENERIC_TABU_SEARCH,
}

# 문자열 리스트를 실제 열거형 값으로 변환
first_solution_strategies_ordered = [
    'GLOBAL_CHEAPEST_ARC',
    'LOCAL_CHEAPEST_ARC',
    'PATH_CHEAPEST_ARC',
    'EVALUATOR_STRATEGY',
    'ALL_UNPERFORMED',
    'BEST_INSERTION',
    'PARALLEL_CHEAPEST_INSERTION',
    'LOCAL_CHEAPEST_INSERTION',
    'SAVINGS',
    'SWEEP',
    'FIRST_UNBOUND_MIN_VALUE',
    'CHRISTOFIDES'
]

local_search_metaheuristics_ordered = [
    'GREEDY_DESCENT',
    'GUIDED_LOCAL_SEARCH',
    'SIMULATED_ANNEALING',
    'TABU_SEARCH',
    'GENERIC_TABU_SEARCH'
]
matrix_obj=np.zeros((len(first_solution_strategies_ordered),len(local_search_metaheuristics_ordered)))
matrix_dis=np.zeros((len(first_solution_strategies_ordered),len(local_search_metaheuristics_ordered)))
matrix_load=np.zeros((len(first_solution_strategies_ordered),len(local_search_metaheuristics_ordered)))
# 반복문에서 main 함수 실행
final_route=[]
for strategy in range(len(first_solution_strategies_ordered)):
    for heuristic in range(len(local_search_metaheuristics_ordered)):
        first_solution_strategy = first_solution_strategy_mapping[first_solution_strategies_ordered[strategy]]
        local_search_metaheuristic = local_search_metaheuristic_mapping[local_search_metaheuristics_ordered[heuristic]]
        print(f"{strategy} {heuristic} :", end="")
        obj,dis,load,route=main(first_solution_strategy, local_search_metaheuristic)
        if obj=="no result":
            pass
        else:
            matrix_obj[strategy][heuristic]=obj
            matrix_dis[strategy][heuristic]=dis
            matrix_load[strategy][heuristic]=load
            final_route.append(route)
        

0 0 :0 1 :0 2 :0 3 :0 4 :1 0 :1 1 :1 2 :1 3 :1 4 :2 0 :Total distance of all routes: 155450m
2 1 :Total distance of all routes: 153153m
2 2 :Total distance of all routes: 155450m
2 3 :Total distance of all routes: 154840m
2 4 :Total distance of all routes: 155450m
3 0 :3 1 :3 2 :3 3 :3 4 :4 0 :4 1 :4 2 :4 3 :4 4 :5 0 :5 1 :5 2 :5 3 :5 4 :6 0 :Total distance of all routes: 162600m
6 1 :Total distance of all routes: 154618m
6 2 :Total distance of all routes: 162585m
6 3 :Total distance of all routes: 161006m
6 4 :Total distance of all routes: 162600m
7 0 :Total distance of all routes: 161778m
7 1 :Total distance of all routes: 154304m
7 2 :Total distance of all routes: 161778m
7 3 :Total distance of all routes: 158333m
7 4 :Total distance of all routes: 159858m
8 0 :8 1 :8 2 :8 3 :8 4 :9 0 :9 1 :9 2 :9 3 :9 4 :10 0 :10 1 :10 2 :10 3 :10 4 :11 0 :Total distance of all routes: 157230m
11 1 :Total distance of all routes: 154777m
11 2 :Total distance of all routes: 157230m
11 3 :Total distan

# 알고리즘 구현 결과 

## 1. 목적함수(거리) 결과

In [7]:
df=pd.DataFrame(matrix_dis,index=first_solution_strategies_ordered,columns=local_search_metaheuristics_ordered)
df = df[(df != 0.0).any(axis=1)]
df

,GREEDY_DESCENT,GUIDED_LOCAL_SEARCH,SIMULATED_ANNEALING,TABU_SEARCH,GENERIC_TABU_SEARCH
PATH_CHEAPEST_ARC,155450.0,153153.0,155450.0,154840.0,155450.0
PARALLEL_CHEAPEST_INSERTION,162600.0,154618.0,162585.0,161006.0,162600.0
LOCAL_CHEAPEST_INSERTION,161778.0,154304.0,161778.0,158333.0,159858.0
CHRISTOFIDES,157230.0,154777.0,157230.0,154793.0,157230.0


In [10]:
import re
def jingyu(final_route):
    distances = []
    loads = []
    for route in final_route:
        distance_match = re.search(r'Distance of the route: (\d+)m', route)
        load_match = re.search(r'Load of the route: (\d+)', route)
        if distance_match and load_match:
            distances.append(int(distance_match.group(1)))
            loads.append(int(load_match.group(1)))

    # 결과 출력
    distance_A,distance_B,distance_C,distance_D,distance_E=0,0,0,0,0
    # for i, (distance, load) in enumerate(zip(distances, loads)):
    #     print(f"Vehicle {i}: Distance = {distance}m, Load = {load}")
    for i in range(12):
        if i%5==0: #6100
            distance_A+=distances[i]
        elif i%5==1:#5000
            distance_B+=distances[i]
        #elif i%5==2:#3700
        #    distance_C+=distances[i]
        elif i%5==3:#3000
            distance_D+=distances[i]
        else:#1500
            distance_E+=distances[i]
    
    distance_E+=distances[-1]
    
    return np.array([distance_A,distance_B,distance_C,distance_D,distance_E])
#[6100, 5000, 3000,1500]*2+[6100,5000,3000,1500,1500]

## 2. 알고리즘별 차량별 이동량

In [11]:
lst=[]
for i in final_route:
          lst.append(jingyu(i))
pd.DataFrame(lst,columns=["차량1","차량2","차량3","차량4","차량5"])

,차량1,차량2,차량3,차량4,차량5
0,42064,31851,0,22270,59265
1,43030,32966,0,21712,55445
2,42064,31851,0,22270,59265
3,40005,31560,0,22209,61066
4,42064,31851,0,22270,59265
5,36150,37962,0,29200,59288
6,42410,33241,0,25472,53495
7,36150,37962,0,29200,59273
8,40501,37295,0,23226,59984
9,36150,37962,0,29200,59288


In [14]:
oil_1=1490/5600
oil_2=1490/8200
oil_3=1490/6700
oil_4=1490/10600
oil_5=1490/8400
cost_1=(105963636+4287600)/8/312
cost_2=(84916364 +3857660)/8/312
cost_3=(92380000 + 3153620)/8/312
cost_4=(64863636 + 3119110)/8/312
cost_5=(102835455 + 3041100)/8/312
lst=[]
for i in final_route:
          lst.append(jingyu(i))
df2=pd.DataFrame(lst,columns=["차량1","차량2","차량3","차량4","차량5"])
df2.assign(cost=lambda x:x.차량1*oil_1+x.차량2*oil_2+x.차량3*oil_3+x.차량4*oil_4+x.차량5*oil_5+cost_1+cost_2+cost_5+cost_4)

,차량1,차량2,차량3,차량4,차량5,cost
0,42064,31851,0,22270,59265,180015.3
1,43030,32966,0,21712,55445,179718.9
2,42064,31851,0,22270,59265,180015.3
3,40005,31560,0,22209,61066,179725.5
4,42064,31851,0,22270,59265,180015.3
5,36150,37962,0,29200,59288,180530.4
6,42410,33241,0,25472,53495,179786.6
7,36150,37962,0,29200,59273,180527.7
8,40501,37295,0,23226,59984,180850.6
9,36150,37962,0,29200,59288,180530.4


## 3.알고리즘별 표준편차

In [15]:
pd.set_option('display.float_format', '{:.1f}'.format)
m=np.zeros([len(df.index),len(df.columns)])
for i in range(len(df.index)):
          for j in range(len(df.columns)):
                m[i][j]=np.std(jingyu(final_route[i*5+j]))
pd.DataFrame(m,columns=df.columns,index=df.index)

,GREEDY_DESCENT,GUIDED_LOCAL_SEARCH,SIMULATED_ANNEALING,TABU_SEARCH,GENERIC_TABU_SEARCH
PATH_CHEAPEST_ARC,19795.0,18935.3,19795.0,20117.9,19795.0
PARALLEL_CHEAPEST_INSERTION,19120.1,18070.8,19115.9,19920.8,19120.1
LOCAL_CHEAPEST_INSERTION,19446.8,18381.0,19446.8,19067.4,19191.9
CHRISTOFIDES,18284.7,18000.3,18284.7,19964.6,18284.7


In [27]:
#등수
11100*0.3+138040*0.7

99958.0